# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-13 01:47:43] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-13 01:47:43] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-13 01:47:43] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-13 01:47:45] INFO server_args.py:1643: Attention backend not specified. Use fa3 backend by default.


[2026-01-13 01:47:45] INFO server_args.py:2542: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=96 avail_mem=10.96 GB):  20%|██        | 4/20 [00:00<00:01,  9.31it/s] 

Capturing batches (bs=72 avail_mem=10.91 GB):  30%|███       | 6/20 [00:00<00:01,  8.44it/s]

Capturing batches (bs=32 avail_mem=10.86 GB):  50%|█████     | 10/20 [00:01<00:00, 10.64it/s]

Capturing batches (bs=4 avail_mem=10.82 GB):  75%|███████▌  | 15/20 [00:01<00:00, 15.25it/s]

Capturing batches (bs=1 avail_mem=10.81 GB): 100%|██████████| 20/20 [00:01<00:00, 13.42it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sam and I am a complete beginner with JavaScript programming. I have built a chatbot using server side and client side javascripts. The user can type something in the input field and press the submit button which sends the data to the server. Now, I want to have the chatbot to be able to predict the next word in a sentence after the user types it. Can anyone help me with the code?
Certainly! To achieve this, you will need to build an AI chatbot that can predict the next word in a sentence based on the input you give. One approach is to train a machine learning model using a sequence-to-sequence
Prompt: The president of the United States is
Generated text:  trying to arrive at a meeting with the mayor of a city to negotiate a new agreement. The president and the mayor will be in a room of 5 tables, arranged in 3 rows. Each table has a different number of chairs for the president and the mayor. The president has 7 more chairs than the mayor. How

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career and interests. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about your career and interests. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about your career and interests. What can you tell me about yourself? [Name] is a [job title] at [company name]. I

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French Academy of Sciences, and the French National Library. Paris is a bustling metropolis with a rich cultural heritage and is a major economic and political center in Europe. It is also known for its fashion industry and its role in the French Revolution. The city is also home to many famous landmarks and attractions, including the Louvre, the Eiffel Tower, and the Champs-Élysées. Paris is a city that is a true reflection

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with human intelligence: AI systems will become more integrated with human intelligence, allowing them to learn and adapt to new situations more effectively. This will enable AI to perform tasks that are currently beyond the capabilities of humans, such as playing chess or playing musical instruments.

2. Enhanced natural language processing: AI will continue to improve its ability to understand and respond to natural language, allowing for more sophisticated and nuanced interactions with humans. This will enable AI to better understand and respond to human emotions and preferences, and to provide more personalized and context-aware services.

3.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I am [Age] years old. I am a [Job Title] at [Company Name]. I am currently working on [Job Title] at [Company Name] in [Location]. I have a [Number] of years experience in [Job Title]. I am passionate about [occupation], and I believe that our company values [value] and wants to make a positive impact [impact]. I enjoy [a Hobby] and I am always looking for ways to [Something]. I am always up for challenges and looking for opportunities to [something]. I am [positive and energetic]. I am always looking for ways to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is known for its iconic Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. It is the largest city in France and has a rich cultural and historical heritage. Paris is often referred to as the "City of Light" due to its

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

name

],

 and

...

 will

 be

 a

 significant

 increase

In [6]:
llm.shutdown()